In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src/')

In [3]:
from package.logger import setup
setup("INFO")

In [4]:
from package import storage, strtime, key
from package.structs import build
from package.raptor.mcraptor import McRaptor
from package.raptor import bag
from package.gtfs import archive
from typing_extensions import Self


In [5]:
footpaths_dict = storage.read_any_dict("../data/footpaths.pkl")
footpaths_dict = footpaths_dict["footpaths"]

structs_dict = storage.read_any_dict("../data/structs.pkl")
build.validate_structs_dict(structs_dict)


In [6]:
import mcr_py

In [44]:

import shutil
import os
from enum import Enum

from pyrosm.data import get_data
import pyrosm
import geopandas as gpd
import networkx as nx
import osmnx as ox

from package import key, storage, osm
from package.logger import Timed, rlog
from package.graph import igraph, fast_path
from package.footpaths import *

osm_path = ""
city_id = "Koeln"
stops_path = "../data/cleaned/stops.csv"
osm_path = osm_path if osm_path else osm.get_osm_path_from_city_id(city_id)

if not os.path.exists(osm_path) and city_id:
	rlog.info("Downloading OSM data")
	osm.download_city(city_id, osm_path)
else:
	rlog.info("Using existing OSM data")

with Timed.info("Reading stops"):
	stops_df = storage.read_gdf(stops_path)

with Timed.info("Calculating bounding box"):
	combined_geometry = MultiPoint(stops_df.geometry.tolist())
	convex_hull = combined_geometry.convex_hull
	bounding_box = convex_hull.buffer(0.01)


[13:45:51] INFO     Using existing OSM data                                                         ]8;id=732647;file:///tmp/ipykernel_53215/338260996.py\338260996.py]8;;\:]8;id=848677;file:///tmp/ipykernel_53215/338260996.py#25\25]8;;\

           INFO     Reading stops                                                                   ]8;id=180071;file:///tmp/ipykernel_53215/338260996.py\338260996.py]8;;\:]8;id=791693;file:///tmp/ipykernel_53215/338260996.py#27\27]8;;\

           INFO     Reading stops done (0.02 seconds)                                               ]8;id=426853;file:///tmp/ipykernel_53215/338260996.py\338260996.py]8;;\:]8;id=173188;file:///tmp/ipykernel_53215/338260996.py#27\27]8;;\

           INFO     Calculating bounding box                                                        ]8;id=232588;file:///tmp/ipykernel_53215/338260996.py\338260996.py]8;;\:]8;id=11119;file:///tmp/ipykernel_53215/338260996.py#30\30]8;;\

           INFO     Calculating bounding box done (0.01 seconds)                                    ]8;id=165390;file:///tmp/ipykernel_53215/338260996.py\338260996.py]8;;\:]8;id=901805;file:///tmp/ipykernel_53215/338260996.py#30\30]8;;\

In [46]:
bounds = bounding_box.bounds
bounds

(6.959501392113865, 50.96440539211387, 6.982997607886134, 50.98488060788613)

In [47]:
import folium

In [52]:
m = folium.Map(
	location=[bounds[1], bounds[0]],
	zoom_start=12,
)

# plot bounds
folium.Rectangle(
	[[bounds[1], bounds[0]], [bounds[3], bounds[2]]],
	color='black',
	fill=False,
).add_to(m)
for _, row in stops_df.iterrows():
	folium.Marker(
		location=[row['stop_lat'], row['stop_lon']],
		icon=folium.Icon(color='red'),
		popup=row['stop_name'],
	).add_to(m)
m

In [8]:
raw_edges = edges[["u", "v", "length"]].to_dict(orient="records")
raw_edges = [{'u': int(edge['u']), 'v': int(edge['v']), 'weights': [int(edge['length'])]} for edge in raw_edges]
raw_edges[:3]

[{'u': 21063145, 'v': 7151289920, 'weights': [7]},
 {'u': 7151289920, 'v': 10929975, 'weights': [7]},
 {'u': 10929975, 'v': 10853912, 'weights': [132]}]

In [9]:
mcr_py.run_mlc(raw_edges, 21063145)

: 

: 

In [ ]:
class CustomLabel(bag.LabelInterface):
    def __init__(self, time: int, stop: str):
        self.arrival_time = time
        self.travel_time = 0
        self.walking_time = 0
        self.waiting_time = 0

        self.stop = stop

    def __repr__(self):
        return f"Label({self.arrival_time}, t={self.travel_time}, w={self.walking_time}, wait={self.waiting_time})"

    def strictly_dominates(self, other: Self) -> bool:
        return (
            self.arrival_time <= other.arrival_time
            and self.travel_time <= other.travel_time
            and self.walking_time <= other.walking_time
            and self.waiting_time <= other.waiting_time
        )

    def update_along_trip(self, arrival_time: int):
        interval = arrival_time - self.arrival_time
        if interval < 0:
            print(self)
        # assert interval >= 0
        self.arrival_time = arrival_time
        self.travel_time += interval

    def update_along_footpath(self, walking_time: int):
        self.arrival_time = self.arrival_time + walking_time
        self.walking_time += walking_time

    def update_before_route_bag_merge(self, departure_time: int):
        interval = departure_time - self.arrival_time
        if interval < 0:
            print(self)
        # assert interval >= 0
        self.arrival_time = departure_time
        self.waiting_time += interval

    def to_human_readable(self):
        return {
            "arrival_time": strtime.seconds_to_str_time(self.arrival_time),
            "travel_time": strtime.seconds_to_str_time(self.travel_time),
            "walking_time": strtime.seconds_to_str_time(self.walking_time),
            "waiting_time": strtime.seconds_to_str_time(self.waiting_time),
        }

    def copy(self: Self) -> Self:
        label = CustomLabel(self.arrival_time, self.stop)

        label.travel_time = self.travel_time
        label.walking_time = self.walking_time
        label.waiting_time = self.waiting_time

        return label


In [ ]:
label_class = CustomLabel

In [ ]:
mcraptor = McRaptor(structs_dict, footpaths_dict, 2, 60, {}, {}, label_class)
res = mcraptor.run("818", "", "15:00:00")

TypeError: CustomLabel.update_before_route_bag_merge() takes 2 positional arguments but 3 were given

In [ ]:
from rich.pretty import pprint
import pandas as pd


In [ ]:
df = pd.DataFrame.from_dict(
    {
        (outer_key, inner_key): values
        for outer_key, inner_dict in res.items()
        for inner_key, values in enumerate(inner_dict)
    }
).T

df = df.reset_index(names=["stop_id", "idx"])

In [ ]:
df.groupby("stop_id").size().sort_values(ascending=False)

stop_id
180     238
1098    221
6827    219
1097    218
5862    196
       ... 
316      24
343      24
317      23
819      22
818      20
Length: 299, dtype: int64

In [ ]:
df[df["stop_id"] == "818"]

,stop_id,idx,arrival_time,travel_time,walking_time,waiting_time
11415,818,0,17:34:04,-1:59:00,02:29:00,00:06:04
11416,818,1,17:36:04,-1:59:00,02:12:44,00:24:20
11417,818,2,17:36:04,-1:58:00,02:20:16,00:17:48
11418,818,3,17:46:04,-1:55:00,02:32:42,00:18:22
11419,818,4,17:36:04,-1:57:00,02:23:58,00:15:06
11420,818,5,17:33:39,-1:59:00,02:28:17,00:06:22
11421,818,6,17:34:56,-1:56:00,02:29:50,00:09:06
11422,818,7,17:33:01,-1:59:00,02:18:55,00:15:06
11423,818,8,17:34:56,-1:58:00,02:34:52,00:02:04
11424,818,9,17:44:56,-1:54:00,02:38:34,00:12:22


In [ ]:
print(strtime.seconds_to_str_time(57900)) # new arrival time # 61
print(strtime.seconds_to_str_time(57960)) # old arrival tiem # 60
# 36-131-001-124.2.22:151500-45-256_16EC2130-92AF-4579-ACDF-AFF1013D6778

16:05:00
16:06:00


In [ ]:
times_by_stop_by_trip = structs_dict["times_by_stop_by_trip"]
t = times_by_stop_by_trip["36-131-001-124.2.22:151500-45-256_16EC2130-92AF-4579-ACDF-AFF1013D6778"]["60"][0]
print(strtime.seconds_to_str_time(t)) # 60

16:06:00


In [ ]:
tt = times_by_stop_by_trip[
    "36-131-001-124.2.22:151500-45-256_16EC2130-92AF-4579-ACDF-AFF1013D6778"
]
sorted(
    [
        (key, (list(strtime.seconds_to_str_time(t) for t in values)))
        for key, values in tt.items()
    ],
    key=lambda x: x[1][0],
)


[('116', ['15:32:00', '15:32:00']),
 ('111', ['15:34:00', '15:34:00']),
 ('109', ['15:35:00', '15:35:00']),
 ('106', ['15:38:00', '15:38:00']),
 ('120', ['15:41:00', '15:41:00']),
 ('73', ['15:57:00', '15:57:00']),
 ('72', ['15:59:00', '15:59:00']),
 ('57', ['16:01:00', '16:02:00']),
 ('61', ['16:05:00', '16:05:00']),
 ('60', ['16:06:00', '16:06:00']),
 ('62', ['16:07:00', '16:07:00']),
 ('56', ['16:09:00', '16:09:00']),
 ('59', ['16:10:00', '16:10:00']),
 ('58', ['16:11:00', '16:11:00']),
 ('54', ['16:14:00', '16:14:00']),
 ('916', ['16:15:00', '16:15:00']),
 ('159', ['16:16:00', '16:16:00']),
 ('152', ['16:18:00', '16:18:00']),
 ('157', ['16:20:00', '16:20:00']),
 ('162', ['16:22:00', '16:22:00'])]